In [2]:
pip install transformers huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -q openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 22.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.0 MB/s eta 0:00:000:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.4 MB/s eta 0:00:00:00:0100:01


In [4]:
import json
import whisper
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import librosa
import re
import os

In [5]:
from huggingface_hub import login
login(new_session=False)

## Modelli

### Whisper Medium

In [6]:
model_transcription = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [01:04<00:00, 23.6MiB/s]


### Gemma-2-2b-it

In [7]:
model_name = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model_extraction = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Funzioni di utilità

In [8]:
def transcribe_audio(file_path):

    audio, sr = librosa.load(file_path, sr=16000, mono=True)
    result = model_transcription.transcribe(audio)
    text = result["text"]

    # Regex con ignorecase per maggiore flessibilità
    text = re.sub(r"\bmilligrams?\s+per\s+deciliter\b", "mg/dl", text, flags=re.IGNORECASE)
    text = re.sub(r"\bmilligrammi?\s+per\s+decilitro\b", "mg/dl", text, flags=re.IGNORECASE)
    text = re.sub(r"\bmg?\s+per\s+decilitro\b", "mg/dl", text, flags=re.IGNORECASE)
    text = re.sub(r"\bmillimeters?\s+of\s+mercury\b", "mmHg", text, flags=re.IGNORECASE)
    text = re.sub(r"\bmillimetri?\s+di\s+merc[ur]io\b", "mmHg", text, flags=re.IGNORECASE)
    text = re.sub(r"\bmm?\s+di\s+merc[ur]io\b", "mmHg", text, flags=re.IGNORECASE)

    text = re.sub(r"\b(\d+)\s+over\s+(\d+)\b", r"\1/\2", text, flags=re.IGNORECASE)
    text = re.sub(r"\b(\d+)\s+su\s+(\d+)\b", r"\1/\2", text, flags=re.IGNORECASE)
    text = re.sub(r"\b(\d+)\s+slash\s+(\d+)\b", r"\1/\2", text, flags=re.IGNORECASE)

    text = re.sub(r"\bbeats?\s+per\s+minute\b", "bpm", text, flags=re.IGNORECASE)
    text = re.sub(r"\bbattiti?\s+al\s+minuto\b", "bpm", text, flags=re.IGNORECASE)

    text = re.sub(r"\bdegrees?\s+celsius\b", "°C", text, flags=re.IGNORECASE)
    text = re.sub(r"\bgradi?\s+celsius\b", "°C", text, flags=re.IGNORECASE)

    text = re.sub(r"\b(\d+)\s+percent\b", r"\1%", text, flags=re.IGNORECASE)
    text = re.sub(r"\bper\s+cento\b", "%", text, flags=re.IGNORECASE)


    return text.strip()

def extract_all(text):
    prompt = f"""Extract the required information in JSON format from the following clinical text:
    {text}
    ----
    Requirements:
    - Translate fields and values into the same language as the input. any spelling and grammar errors
    - Keep the JSON compact.
    - For fields that are not explicitly mentioned in the text, return an empty string "".

    required information:
        - first_name: name of patient
        - last_name:
        - access_mode: Specify how the patient arrived
        - birth_date:
        - birth_place:
        - age:
        - gender: 
        - residence_city:
        - residence_address: 
        - phone:
        - skin_state:
        - consciousness_state:
        - pupils_state:
        - respiratory_state:
        - history:
        - medications_taken:
        - symptoms:
        - heart_rate:
        - oxygenation:
        - blood_pressure:
        - temperature:
        - blood_glucose:
        - medical_actions:
        - assessment:
        - plan:
        - triage_code: 

    JSON:
    """

    # Tokenizza input + genera output
    inputs = tokenizer(prompt, return_tensors="pt").to(model_extraction.device)

    outputs = model_extraction.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decodifica la risposta generata (in stringa)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

def extract_json_block(text):
    start = text.find('{')
    if start == -1:
        return None

    depth = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            depth += 1
        elif text[i] == '}':
            depth -= 1
            if depth == 0:
                return text[start:i+1]  # JSON completo
    return None  # JSON non chiuso correttamente

def parse_generated_json(generated_text: str):
    """
    Cerca e analizza un blocco JSON all'interno del testo generato dal modello.

    Args:
        generated_text (str): Il testo completo generato dal modello, contenente un blocco JSON.

    Returns:
        dict | None: Il dizionario Python se il parsing ha successo, altrimenti None.
    """
    json_str = extract_json_block(generated_text)

    if json_str:
        try:
            patient_info = json.loads(json_str)
            print("📄 JSON estratto e parsato:")
            return patient_info
        except json.JSONDecodeError as e:
            print("❌ Errore nel parsing JSON:", e)
            return None
    else:
        print("⚠️ Nessun JSON trovato nella risposta generata:")
        return None

## Elaborazione

In [9]:
# Percorso alla cartella dei file audio
input_dir = "/kaggle/input/dataset"
output_dir = "/kaggle/working"

# Ciclo su ogni file nella cartella
for nome_file in os.listdir(input_dir):
    if nome_file.endswith((".mp3", ".wav", ".m4a")):
        percorso_input = os.path.join(input_dir, nome_file)

        print(f"\n🧠 Elaborazione: {nome_file}")

        # Trascrizione e parsing
        text = transcribe_audio(percorso_input)
        print(text)

        extracted_text = extract_all(text)
        extracted_dict = parse_generated_json(extracted_text)
        print(extracted_dict)

        # Salvataggio JSON
        nome_base = os.path.splitext(nome_file)[0]
        nome_file_json = f"{nome_base}.json"
        percorso_output = os.path.join(output_dir, nome_file_json)

        with open(percorso_output, 'w', encoding='utf-8') as f:
            json.dump(extracted_dict, f, ensure_ascii=False, indent=4)

        print(f"💾 File salvato in: {percorso_output}")


🧠 Elaborazione: report completo inglese mio.mp3
Patient Michael Thompson was born on November 2nd 1972 in Philadelphia. He is 52 years old man residing at 22 Rose Hill Lane, Pennsylvania. He was transported by ambulance. Phone number 02-98-76-54-3. The patient was found unconscious in critical condition. He presented with a loss of consciousness and absent respiration. Skin appeared pale and pupils were anisocoric. The patient presents with oxygen saturation of 88%, a heart rate of 110 bpm, a body temperature of 36.5 degrees and blood pressure of 90/60. Blood glucose is measured at 67. The patient has a medical history of type 2 diabetes mellitus. Currently he is not taking any medications. The patient was assigned a red triage code. A hypoglycemic coma is suspected. The initial treatment plan includes the intravenous administration of glucose with continuous monitoring and immediate transport to the emergency department for further care.
📄 JSON estratto e parsato:
{'first_name': 'Mic

In [7]:
def correct_text(text):
    prompt = f"""
Rewrite the following clinical text by correcting any transcription errors.
    ----
    Requirements:
    - Translate corrected text into the same language as the clinical text.

Clinical text:
{text}

Corrected text:
"""

    # Tokenizza input + genera output
    inputs = tokenizer(prompt, return_tensors="pt").to(model_extraction.device)

    outputs = model_extraction.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decodifica la risposta generata (in stringa)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

In [8]:
# Percorso alla cartella dei file audio
input_dir = "/kaggle/input/prova2"
output_dir = "/kaggle/working"

# Ciclo su ogni file nella cartella
for nome_file in os.listdir(input_dir):
    if nome_file.endswith((".mp3", ".wav", ".m4a")):
        percorso_input = os.path.join(input_dir, nome_file)

        print(f"\n🧠 Elaborazione: {nome_file}")

        # Trascrizione e parsing
        text = transcribe_audio(percorso_input)
        print(text)

        corrected_text = correct_text(text)
        print(corrected_text)


🧠 Elaborazione: 1 - Checkup italiano.mp3
Il paziente Marco Esposito presenta parametri vitali nella norma. Frequenza cardiaca 78 BPM, pressione arteriosa 122/78 MMHg, temperatura corporea 36,7 gradi, saturazione di ossigeno al 98%, glicemia a 96 mg su dl. Riferisce leggera stanchezza ma nessun nuovo sintomo e attualmente in terapia con ceftriaxone per l'infezione delle vioni urinarie. In data odierna è stato eseguito un esame delle urine in attesa dei risultati. La dimissione non è ancora prevista, si prosegue con osservazione clinica.

Rewrite the following clinical text by correcting any transcription errors.
    ----
    Requirements:
    - Translate corrected text into the same language as the clinical text.

Clinical text:
Il paziente Marco Esposito presenta parametri vitali nella norma. Frequenza cardiaca 78 BPM, pressione arteriosa 122/78 MMHg, temperatura corporea 36,7 gradi, saturazione di ossigeno al 98%, glicemia a 96 mg su dl. Riferisce leggera stanchezza ma nessun nuovo s